# Comparativa de resultados
A lo largo de este notebook se cubre la quinta fase de la metodología CRIPS-DM, en este caso la evaluación de los resultados

1. **Comprensión del Negocio (Business Understanding)**
   - Consistente en el entendimiento del objetivo del proyecto.

2. **Comprensión de los Datos (Data Understanding)**
   - Relacionada con la carga y primera evaluación del conjunto de datos. Se divide a su vez en :


3. **Preparación de los Datos (Data Preparation)** 
   - Consistente en la limpieza, preparación y extracción de características de los datos.

4. **Modelado (Modeling)**  
   - Relacionada con la elección del modelo de machine learning y el ajuste hiperparamétrico.

5. <span style="color:#66FF99;">**Evaluación (Evaluation)**  </span> 
   - Evaluación de los resultados obtenidos y comparativa de los mismos emtre modelos.

6. **Implementación (Deployment)**  
   - Integración del modelo de forma que sea accesible para su uso.


## Evaluación 


Tras el entrenamiento de diversos modelos 

Esta sección puede ser dividida en dos subsecciones:

1) Evaluación y comparativa cuantitativa: se comapran resultados entre modelos de acuerdo con funciones de rendimiento

2) Evaluación cualitativa: se estudia como se comporta cada uno de los modelos ante las imágenes, y su relación con los etiquetados del dataset

### 1. Evaluación cuantitativa

En este caso, se ha considerado relevante el estudio de tres métricas de rendimeinto para evaluar el funcionamiento de cada uno de los modelos entrenados

1) 
2) 
3) 

### 2. Evaluación cualitativa

A continuación, se genera la inferencia para cada imagen del dataset y se comprueban no únicamente los resultados del modelo, sino que también su relación con la máscara estipulada por el conjunto de datos seleccionado.
Para ello, hacemos uso de cada una de las pipelines definidas para cada modelo.